In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt

In [47]:
class ContractiveAutoencoder(nn.Module):
    def __init__(self):
        super(ContractiveAutoencoder, self).__init__()
        # Encoder layers
        self.flatten = nn.Flatten()
        self.encoder_fc1 = nn.Linear(28 * 28, 64)
        self.encoder_fc2 = nn.Linear(64, 32)
        self.bottleneck = nn.Linear(32, 16)  # bottleneck layer

        # Decoder layers
        self.decoder_fc1 = nn.Linear(16, 32)
        self.decoder_fc2 = nn.Linear(32, 64)
        self.output_fc = nn.Linear(64, 28 * 28)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.encoder_fc1(x))
        x = F.relu(self.encoder_fc2(x))
        h = F.relu(self.bottleneck(x))  # Latent representation (bottleneck)

        x = F.relu(self.decoder_fc1(h))
        x = F.relu(self.decoder_fc2(x))
        x = self.output_fc(x)
        x = x.view(x.size(0), 1, 28, 28)

        return x, h

In [48]:
def contractive_loss(x, x_reconstructed, h, model, Lambda=100):
    # Reconstruction loss
    mse_loss = F.mse_loss(x_reconstructed, x, reduction='sum')
    
    # Contractive loss
    dh = h * (1 - h)
    W = model.bottleneck.weight
    contractive = Lambda * torch.sum(dh ** 2 * torch.sum(W ** 2, dim=1))

    total_loss = mse_loss + contractive
    return total_loss

In [49]:
def evaluate_CAE(testloader, model, device, Lambda):
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for x_batch, label in testloader:
            x_batch = x_batch.to(device)
            x_reconstructed, h = model(x_batch)
            loss = contractive_loss(x_batch, x_reconstructed, h, model, Lambda)
            test_loss += loss.item()

    avg_test_loss = test_loss / len(testloader.dataset)
    model.train()  # Switch back to training mode
    return avg_test_loss

In [50]:
def train_CAE(trainloader, testloader, trainset, testset, num_epochs, batch_size, model, optimizer, device, log_dir="./tensorboard_logs"):
    writer = SummaryWriter(log_dir)
    Lambda = 100  # Contractive regularization weight
    dataiter = iter(testloader)

    for epoch in range(num_epochs):
        train_loss = 0.0
        for i, (x_batch, label) in enumerate(trainloader):
            x_batch = x_batch.to(device)

            # Forward pass
            x_reconstructed, h = model(x_batch)

            # Compute loss and gradients
            loss = contractive_loss(x_batch, x_reconstructed, h, model, Lambda)
            train_loss += loss.item()
            

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        avg_train_loss = train_loss / len(trainloader.dataset)
        writer.add_scalar("CAE Loss/Train", avg_train_loss, epoch)

        # Log validation loss
        test_loss = evaluate_CAE(testloader, model, device, Lambda)
        writer.add_scalar("CAE Loss/Test", test_loss, epoch)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item()}")

In [55]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ContractiveAutoencoder().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
batch_size = 32
num_epochs = 5
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

train_CAE(trainloader, testloader, trainset, testset, num_epochs, batch_size, model, optimizer, device)

Epoch [1/5], Loss: 643.9605102539062
Epoch [2/5], Loss: 654.4360961914062
Epoch [3/5], Loss: 600.0396118164062
Epoch [4/5], Loss: 547.0094604492188
Epoch [5/5], Loss: 653.9487915039062


In [52]:
class AE(torch.nn.Module):
    def __init__(self):
        super().__init__()

        # Building an linear encoder with Linear
        # layer followed by Relu activation function
        # 784 ==> 9
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(28 * 28, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 18),
            torch.nn.ReLU(),
            torch.nn.Linear(18, 9)
        )

        # Building an linear decoder with Linear
        # layer followed by Relu activation function
        # The Sigmoid activation function
        # outputs the value between 0 and 1
        # 9 ==> 784
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(9, 18),
            torch.nn.ReLU(),
            torch.nn.Linear(18, 36),
            torch.nn.ReLU(),
            torch.nn.Linear(36, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 28 * 28),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [54]:
def train_AE(loader, device, model, loss_function, optimizer, epochs, log_dir="./tensorboard_logs"):
    writer = SummaryWriter(log_dir=log_dir)
    
    for epoch in range(epochs):
        running_loss = 0.0  # Accumulate loss for averaging

        for batch_idx, (data, _) in enumerate(loader):  # Unpack the data and ignore labels
            # Flattening the image
            image = data.reshape(-1, 28 * 28).to(device)

            # Output of Autoencoder
            reconstructed = model(image)

            # Calculating the loss function
            loss = loss_function(reconstructed, image)

            # Zero gradients, backward pass, and optimizer step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Accumulate loss for this batch
            running_loss += loss.item()

            # Log the loss to TensorBoard for each batch
            global_step = epoch * len(loader) + batch_idx
            writer.add_scalar("AE Loss/Batch", loss.item(), global_step)

        # Calculate and log average loss per epoch
        average_loss = running_loss / len(loader)
        writer.add_scalar("AE Loss/Epoch", average_loss, epoch)
        
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {average_loss:.4f}")


In [56]:
tensor_transform = transforms.ToTensor()
dataset = datasets.MNIST(root = "./data", train = True, download = True, transform = tensor_transform)
loader = torch.utils.data.DataLoader(dataset = dataset, batch_size = 30, shuffle = True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AE().to(device)

loss_function = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

epochs = 5

train_AE(loader, device, model, loss_function, optimizer, epochs)

Epoch [1/5], Loss: 0.0544
Epoch [2/5], Loss: 0.0337
Epoch [3/5], Loss: 0.0282
Epoch [4/5], Loss: 0.0263
Epoch [5/5], Loss: 0.0252


In [ ]:
# tensorboard --logdir=c:\Users\AjayK\git\VAE\tensorboard_logs

In [ ]:
pip install "gymnasium[atari, accept-rom-license]" opencv-python "stable-baselines3" pygame matplotlib
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

SyntaxError: invalid syntax (3158742353.py, line 1)

In [ ]:
import gymnasium as gym
from gymnasium import spaces

import pygame

import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch.autograd import Variable
from torchvision.transforms import v2

from stable_baselines3 import PPO

In [ ]:
env = gym.make("ALE/Pong-v5", render_mode="human")
env.obs_type = "grayscale"
observation, info = env.reset()
Image.fromarray(observation)
device = torch.device("cpu")
height = observation.shape[0]
width = observation.shape[1]
channels = observation.shape[2]
process = transforms.Compose([
    transforms.ToTensor()
])
observation = process(observation)
observation = observation.unsqueeze(0).to(device)

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        input = input.view(input.size(0), -1)
        return input

In [ ]:
class UnFlatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), 256, 11, 8)

In [ ]:
class VAE(nn.Module):
    def __init__(self, image_channels=3, height=height, width=width):
        super(VAE, self).__init__()

        self.height = height
        self.width = width
        
        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, 32, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2),
            nn.ReLU(),
            Flatten()
        )
        
        self.decoder = nn.Sequential(
            UnFlatten(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(32, image_channels, kernel_size=4, stride=2),
            nn.Sigmoid(),
        )

In [ ]:
def forward(self, x):
        latent = self.encoder(x)
        return self.decoder(latent), latent.to("cpu").detach().numpy()

In [ ]:
# Model Initialization
VAE_model = VAE().to(device)
 
# Validation using MSE Loss function
loss_function = torch.nn.MSELoss()
 
# Using an Adam Optimizer
optimizer = torch.optim.Adam(VAE_model.parameters())

losses = []
for _ in range(10000):
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)

    observation = process(observation).unsqueeze(0).to(device)
    reconstruction, latent = VAE_model(observation)
    loss = loss_function(reconstruction, observation[:, :, :206, :158])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    losses.append(loss)

    if terminated or truncated:
        observation, info = env.reset()
env.close()